# Analysis of E. Coli predictor markers
* Predictor alignment on 52677 genomes

In [1]:
# Essential Libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Data set
predictor_df = pd.read_csv('gene_presence_report.csv', index_col=0)

In [ ]:
# Num of hits per predictor marker
per_marker_df = predictor_df.sum(axis=0)
print(per_marker_df)
plt.figure()
per_marker_df.plot(kind='bar')
plt.show()

In [ ]:
# Summary of total hits per genome
total_hit_s = predictor_df.sum(axis=1)
total_hit_s.describe()

In [ ]:
# Summary of genome count per number of hits
val_count_s = total_hit_s.value_counts().sort_index()
print(val_count_s)
val_count_s.plot(kind='bar')
plt.show()

In [ ]:
# List genomes that do not meet threshold (3)
bad_genomes_s = total_hit_s[total_hit_s.isin(range(3))].index
bad_genomes_df = pd.DataFrame(bad_genomes_s)
bad_genomes_df.to_csv('bad_genomes.csv')

In [ ]:
# Choose 10 at random
'''
import random
samples = random.sample(bad_genomes, 10)
'''
samples = [
    'ESC_BA9048AA_AS', 'ESC_BA4307AA_AS', 'ESC_KA3487AA_AS',
    'ESC_GA7863AA_AS', 'ESC_CA5285AA_AS', 'ESC_CA4089AA_AS',
    'ESC_BA7159AA_AS', 'ESC_CA2883AA_AS', 'ESC_AA8910AA_AS',
    'ESC_JA5955AA_AS'
]
print(samples)

### Quick investigation to verify bad genomes
1. Randomly sample 10 of the bad genomes
1. Select the first contig with bp>10000 in each genome
1. Find the highest score match in ncbi blast search


#### Output format below:
* Genome assembly name
    * Contig header
        * Best match genome description

## Output:
* ESC_AA8910AA_AS
    * NODE_98_length_13711_cov_20.0465_ID_195
        * Shigella flexneri 2a str. 301, complete genome
* ESC_BA4307AA_AS
    * NODE_92_length_16268_cov_17.8584_ID_183
        * Shigella sonnei strain CFSAN030807 chromosome, complete genome
* ESC_BA7159AA_AS
    * NODE_14_length_40012_cov_6.28077_ID_27
        * Select seq gb|CP011511.1|	Shigella boydii strain ATCC 9210, complete genome
* ESC_BA9048AA_AS
    * NODE_4_length_73392_cov_10.6272_ID_7
        * Shigella flexneri 4c strain 0702, complete genome	
* ESC_CA2883AA_AS
    * NODE_37_length_34263_cov_15.3969_ID_73
        * Shigella sonnei strain CFSAN030807 chromosome, complete genome
* ESC_CA4089AA_AS
    * NODE_171_length_7659_cov_13.2305_ID_341
        * Select seq gb|CP023645.1|	Shigella sonnei strain CFSAN030807 chromosome, complete genome
* ESC_CA5285AA_AS
    * NODE_31_length_37642_cov_17.5252_ID_61
        * Select seq gb|CP022457.1|	Shigella sonnei strain 2015C-3566, complete genome
* ESC_GA7863AA_AS
    * NODE_44_length_28919_cov_3.21009_ID_87
        * Shigella sonnei strain CFSAN030807 chromosome, complete genome
* ESC_JA5955AA_AS
    * NODE_110_length_13533_cov_4.26666
        * Shigella boydii strain ATCC 9210, complete genome
* ESC_KA3487AA_AS
    * NODE_107_length_12436_cov_9.79464
        * Shigella flexneri Shi06HN006, complete genome

In [ ]:

# Mash identification to find best match. Slow (~90 seconds per genome)
'''
import sys, os
from tqdm import tqdm
ectyper_dir = '/home/sam/Projects/ecoli_serotyping'
sys.path.append(ectyper_dir)
from ectyper import speciesIdentification
sample_dir = 'sample'
species_results = {}
for sample in tqdm(samples):
    sample_file = os.path.join(sample_dir, sample+'.fasta')
    species_results[sample]=speciesIdentification.get_species(sample_file)
'''
species_results = {'ESC_BA9048AA_AS': '[310 seqs] NZ_AZPN01000001.1 Shigella flexneri 2002021 2002021_0001, whole genome shotgun sequence [...]\n', 'ESC_BA4307AA_AS': '[407 seqs] NZ_CWTD01000001.1 Shigella sonnei strain 38101ss_1, whole genome shotgun sequence [...]\n', 'ESC_KA3487AA_AS': '[336 seqs] NZ_LPSY01000009.1 Shigella boydii strain 100706 100706_10, whole genome shotgun sequence [...]\n', 'ESC_GA7863AA_AS': '[421 seqs] NZ_CXEQ01000001.1 Shigella sonnei strain 20052631_1361398, whole genome shotgun sequence [...]\n', 'ESC_CA5285AA_AS': '[394 seqs] NZ_CWWB01000001.1 Shigella sonnei strain 201403955_1, whole genome shotgun sequence [...]\n', 'ESC_CA4089AA_AS': '[347 seqs] NZ_CXAT01000001.1 Shigella sonnei strain Sh62542_401057, whole genome shotgun sequence [...]\n', 'ESC_BA7159AA_AS': '[924 seqs] NZ_LPTC01000009.1 Shigella boydii strain 603122 603122_10, whole genome shotgun sequence [...]\n', 'ESC_CA2883AA_AS': '[443 seqs] NZ_CXGM01000001.1 Shigella sonnei strain H112920489, whole genome shotgun sequence [...]\n', 'ESC_AA8910AA_AS': 'NZ_CP004056.1 Shigella flexneri 2003036, complete genome\n', 'ESC_JA5955AA_AS': '[644 seqs] NZ_MIIX01000001.1 Shigella sp. FC1708 NODE_100_length_14432_cov_26.0267_ID_199, whole genome shotgun sequence [...]\n'}

In [ ]:
from pprint import pprint
pprint(species_results)

# Cross-examination with mash identification
* Mash identification on 5844 serotyped genomes

In [ ]:
# Load data from result csv file
species_result_file = 'species_results.csv'
mash_df = pd.read_csv(species_result_file)
mash_df = pd.concat([mash_df['genome name'].str.replace('(.fasta)',''), mash_df['species']], axis=1)
mash_df.describe()

In [ ]:
# Find all rows that is neither Escherichia nor Shigella
s1 = mash_df['species'].str.find('Escherichia')!=-1
s2 = mash_df['species'].str.find('Shigella')!=-1
mash_df[~s1 & ~s2]

In [ ]:
# Merge mash result with alignment hits result (inner join)
total_hit_df = pd.DataFrame(total_hit_s, columns=['num of hits'])
merge_df = mash_df.merge(total_hit_df, left_on='genome name', right_index=True, how='inner')
merge_df.describe()

In [ ]:
# Find all rows that is not Escherichia by predictor alignment
non_ecoli_merge_df1 = merge_df[merge_df['num of hits'].isin((range(3)))]
non_ecoli_merge_df1
# Explanation: Predictor alignment false negative rate = 16/18

In [ ]:
# Find all rows that is not Escherichia based on mash identification
non_ecoli_merge_df2 = merge_df[merge_df['species'].str.find('Escherichia')==-1]
non_ecoli_merge_df2
# Explanation: Mash identification false negative rate = 10/12

In [ ]:
genome_dir = '/home/sam/Projects/MoreSerotype/temp/genomes'
from Bio import SeqIO
import os
from tqdm import tqdm
non_ecoli_genomes = list(non_ecoli_merge_df1[merge_df['species'].str.find('Escherichia')!=-1].get('genome name'))
genome_infos = []
for genome in tqdm(non_ecoli_genomes):
    genome_file = os.path.join(genome_dir, genome+'.fasta')
    entry = {
        'genome name': genome
    }
    l = []
    for record in SeqIO.parse(genome_file, 'fasta'):
        l.append(str(len(record.seq)))
    entry['lengths'] = ','.join(l)
    entry['count'] = len(l)
    genome_infos.append(entry)
quality_df = pd.DataFrame(genome_infos)
quality_df = quality_df.set_index('genome name')
quality_df.median()

In [ ]:
genome_dir = '/home/sam/moria/enterobase_db'
from Bio import SeqIO
from tqdm import tqdm
non_ecoli_genomes = list(non_ecoli_merge_df2[~merge_df['num of hits'].isin((range(3)))].get('genome name'))
genome_infos = []
for genome in tqdm(non_ecoli_genomes):
    genome_file = os.path.join(genome_dir, genome+'.fasta')
    entry = {
        'genome name': genome
    }
    l = []
    for record in SeqIO.parse(genome_file, 'fasta'):
        l.append(str(len(record.seq)))
    entry['lengths'] = ','.join(l)
    entry['count'] = len(l)
    genome_infos.append(entry)
quality_df = pd.DataFrame(genome_infos)
quality_df = quality_df.set_index('genome name')
quality_df.median()

In [ ]:
genome_dir = '/home/sam/moria/enterobase_db'
from Bio import SeqIO
from tqdm import tqdm
import random
all_genomes = list(predictor_df.index)
samples = random.sample(all_genomes, 10)
genome_infos = []
for genome in tqdm(samples):
    genome_file = os.path.join(genome_dir, genome+'.fasta')
    entry = {
        'genome name': genome
    }
    l = []
    for record in SeqIO.parse(genome_file, 'fasta'):
        l.append(str(len(record.seq)))
    entry['lengths'] = ','.join(l)
    entry['count'] = len(l)
    genome_infos.append(entry)
quality_df = pd.DataFrame(genome_infos)
quality_df = quality_df.set_index('genome name')
quality_df.median()

## Intrepretation
* Both **marker prediction** and **mash prediction** have high accuracy for correct predictions but have high false negative rate as well
* **marker prediction** have false negative prediction when they quality of genome is poor (indicated by high contig count)
* **mash prediction**'s false negative rate appears to be independent of genome quality

## Expand columns

In [ ]:
strains = pd.read_json('strains.json')
report = pd.read_csv('gene_presence_report.csv')
report.describe()

In [ ]:
new_report = report.merge(
    strains[['assembly_barcode', 'serotype', 'species']],
    left_on='genome_name', right_on='assembly_barcode', how='left'
)
new_report.columns = [
    'genome name', '1436893830000|3159571',
    '1436893909000|3159808', '2873786891000|3159389', '2873787160000|3160196',
    '4310679577000|3158082', '4310679772000|3158667', '4310679831000|3158844',
    '4310680254000|3160113', '4310680315000|3160296', '4310680399000|3160548',
    'assembly_barcode', 'enterobase serotype', 'enterobase species'
]
new_report['hit count'] = new_report[marker_cols].sum(axis=1)
new_report = new_report[[
    'genome name', 'hit count',
    'enterobase serotype', 'enterobase species',
    '1436893830000|3159571',
    '1436893909000|3159808', '2873786891000|3159389', '2873787160000|3160196',
    '4310679577000|3158082', '4310679772000|3158667', '4310679831000|3158844',
    '4310680254000|3160113', '4310680315000|3160296', '4310680399000|3160548',
]]
new_report.to_csv('new_report.csv', index=False)
new_report

In [4]:
report = pd.read_csv('new_report.csv', index_col='genome name')
mash_csv = pd.read_csv('mash_for_lowhitecoli.csv')[['barcode', 'species']]

In [37]:
new_report = report.merge(mash_csv, left_index=True, right_on='barcode', how='left')
new_cols = [
    'hit count', 'enterobase serotype', 'enterobase species',
    '1436893830000|3159571', '1436893909000|3159808',
    '2873786891000|3159389', '2873787160000|3160196',
    '4310679577000|3158082', '4310679772000|3158667',
    '4310679831000|3158844', '4310680254000|3160113',
    '4310680315000|3160296', '4310680399000|3160548', 'barcode', 'mash species'
]
new_report.columns = new_cols
new_cols = [
    'barcode', 'hit count', 'enterobase serotype',
    'enterobase species','mash species',
    '1436893830000|3159571', '1436893909000|3159808',
    '2873786891000|3159389', '2873787160000|3160196',
    '4310679577000|3158082', '4310679772000|3158667',
    '4310679831000|3158844', '4310680254000|3160113',
    '4310680315000|3160296', '4310680399000|3160548'
]
new_report = new_report[new_cols]
new_report = new_report.set_index(keys='barcode')
new_report.to_csv('new_report2.csv')

In [20]:
new_report[new_report['species'].notnull()].columns

Index(['hit count', 'enterobase serotype', 'enterobase species',
       '1436893830000|3159571', '1436893909000|3159808',
       '2873786891000|3159389', '2873787160000|3160196',
       '4310679577000|3158082', '4310679772000|3158667',
       '4310679831000|3158844', '4310680254000|3160113',
       '4310680315000|3160296', '4310680399000|3160548', 'barcode', 'species'],
      dtype='object')